# Visualising the Earth's normal modes

This file contains the code for doing the following:

1. Computing surface vectors for the Earth's normal modes

2. Using these to create 3D visualisations which show the Earth's surface oscillating in its normal modes

3. Importing eigenfunction databases calculated using Mineos, and plotting the eigenfunctions

4. Using these eigenfunctions to calculate the displacements for the entire Earth, and creating 3D visualisations using Mineos

### Importing the required libraries and packages

In [1]:
# NOTE: all of these packages must be installed on your machine for the code to work
# all of the packages except mayavi can be installed easily using pip
# the installation for mayavi is explained in a different file
import numpy as np
import math
import matplotlib.pyplot as plt
from mayavi import mlab
import os
import imageio.v2 as imageio # for saving mayavi animations
%gui qt

### Computing the spherical harmonics

In [2]:
# before we can compute spherical harmonics, 
# we need a way to compute derivatives of different orders of a polynomial, 
# and a way to compute Legendre functions

# using symbolic differentiation iteratively
def derivative(y, order): # y must be a np.poly1d instance, which has a deriv method
    for i in range(order):
        y = y.deriv()
    return y

# creating a Legendre function of the given orders, and giving x as the argument
def poly(m, l, x):
    first_bracket = ((1-x**2)**(m/2))/((2**l)*math.factorial(l))
    y = np.poly1d([1, 0, -1])**l # constructing the polynomial (x**2 - 1)**l
    deriv = derivative(y, order=l+m) # taking its derivative of order l + m
    return first_bracket * deriv(x)
# we can give cos(theta) as the argument
# then, what is returned here can be used in the spherical harmonic

# computing the spherical harmonic of desired m and l orders
def spherical_harmonic(m, l, phi, theta):
    first_bracket = (-1)**m
    second_bracket = ( ((2*l + 1)/(4*np.pi)) * ((math.factorial(l - m))/(math.factorial(l + m))) )**(1/2)
    third_bracket = poly(m, l, np.cos(theta)) * np.e**(1j*m*phi)
    
    return first_bracket*second_bracket*third_bracket

### Visualising the spherical harmonics

In [3]:
def plot_sph_harm(m, l):
    phi, theta = np.mgrid[0.0:2.0*np.pi-0.0:100j, 0.0:np.pi-0.0:100j] # phi = azimuthal, theta = polar 

    # defining the cartesian coordinates for plotting
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    Y_m_l = spherical_harmonic(m, l, phi, theta)

    mlab.clf()
    surface_plot = mlab.mesh(x, y, z, scalars = np.real(Y_m_l), colormap='seismic')

In [4]:
# an example showing how the functions above can be used

plot_sph_harm(0, 3)

### Computing the surface vectors (**R**, **S** and **T**)

In [6]:
# NOTE: These functions only return the real parts of the vectors, discarding the imaginary parts
# all 3 functions return the vectors in the following format: [radial component, polar component, azimuthal component]

def R_(m, l, phi, theta):
    r_comp = np.real(spherical_harmonic(m, l, phi, theta))

    phi_comp = np.zeros_like(phi)

    theta_comp = np.zeros_like(phi)

    R_mode = np.array([r_comp, theta_comp, phi_comp])

    return R_mode

def S_(m, l, phi, theta):
    
    r_comp = np.zeros_like(phi)

    theta_comp = np.real(
        (-1)**(m+1)*
            (
                ((2*l + 1)/(4*np.pi)) * ((math.factorial(l - m))/(math.factorial(l + m))) 
            )**(1/2) *
        np.e**(1j * m * phi) *
        np.sin(theta) *
        1/(2**l * math.factorial(l)) * 
            (
                (1 - np.cos(theta)**2)**(m/2) * (derivative(np.poly1d([1, 0, -1]) ** l, order=(l + m + 1)))(np.cos(theta)) - 
                np.cos(theta) * m * (1 - np.cos(theta)**2)**(m/2 - 1) * 
                (derivative(np.poly1d([1, 0, -1]) ** l, order=(l + m))(np.cos(theta)))
            )
    )
    if l != 0:
        theta_comp = theta_comp * (1/np.sqrt(l*(l+1)))
    
    phi_comp = np.real(1/np.sin(phi) * spherical_harmonic(m, l, phi, theta) * 1j*m)
    
    if l != 0:
        phi_comp = phi_comp * (1/np.sqrt(l*(l+1)))

    S_mode = np.array([r_comp, theta_comp, phi_comp])

    return S_mode

def T_(m, l, phi, theta):
    
    r_comp = np.zeros_like(phi)

    phi_comp = - np.real(
        (-1)**(m+1)*
            (
                ((2*l + 1)/(4*np.pi)) * ((math.factorial(l - m))/(math.factorial(l + m))) 
            )**(1/2) *
        np.e**(1j * m * phi) *
        np.sin(theta) *
        1/(2**l * math.factorial(l)) * 
            (
                (1 - np.cos(theta)**2)**(m/2) * (derivative(np.poly1d([1, 0, -1]) ** l, order=(l + m + 1)))(np.cos(theta)) - 
                np.cos(theta) * m * (1 - np.cos(theta)**2)**(m/2 - 1) * 
                (derivative(np.poly1d([1, 0, -1]) ** l, order=(l + m))(np.cos(theta)))
            )
    )
    
    if l != 0:
        phi_comp = phi_comp * (1/np.sqrt(l*(l+1)))

    theta_comp = np.real(1/np.sin(phi) * spherical_harmonic(m, l, phi, theta) * 1j*m)

    if l != 0:
        theta_comp = theta_comp * (1/np.sqrt(l*(l+1)))

    T_mode = np.array([r_comp, theta_comp, phi_comp]) 

    return T_mode

### Visualising the spheroidal modes (surface only)

In [12]:
def surf_anim_S(m, l, phi_no, theta_no, time_max, time_interval, omega=1, shape='sphere', savefig=False, points=True, mesh=True, vectors=False, save_path='./', view=(0, 90)):
    # phi_no and theta_no is the desired resoluion of the animated mesh in the azimuthal and polar directions respectively
    # time_max is the total time of the animation
    # time_interval is the resolution of the time axis
    # omega (default = 1) is the frequency of the animation (the eigenfrequency of the mode can be used but it is usually not suited for visualisation)
    # shape: sphere, hemisphere, 3/4th of a sphere, 1/4th of a sphere
    # savefig: whether or not the animation is to be saved
    # save_path: the path at which to save the .gif for the animation as well as the images for all the frames (temporarily)
    # view = (azimuthal angle, elevation angle): allows one to set the viewing angle for the animation, which can also be adjusted manually in the mayavi window
    # NOTE: if savefig is set to true, the function will create each frame as an image and stitch them all together as a gif, deleting the images, this may take up a lot of space and time for longer or smoother animations

    if savefig:
        images=[]
    
    # creating the mesh on which to calculate the surface vectors        
    if shape=='sphere':
        phi, theta = np.mgrid[0.0001:2.0*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='hemisphere':
        phi, theta = np.mgrid[0.0001:1.0*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    elif shape=='3_quarter':
        phi, theta = np.mgrid[0.0001:1.5*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    elif shape=='1_quarter':
        phi, theta = np.mgrid[0.0001:0.5*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    # creating the time axis for the animation
    time = np.arange(0, time_max, time_interval)
    
    # calculating the x, y, z coordinates of each point 
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    # calculating the output of the surface vectors
    S = S_(m, l, phi, theta)
    R = R_(m, l, phi, theta)
    result = S + R
    
    # the periodic function which introduces the time dependence
    # NOTE: we are only taking the real part of this, so it can be replaced with a cosine
    periodic_f = np.real(np.e**(1j * time) * omega)

    # calculating the x, y, z components as the points are plotted in cartesian coordinates
    x_comp = (result[0] * np.sin(theta) * np.cos(phi) + 
                result[1] * np.cos(theta) * np.cos(phi) - 
                result[2] * np.sin(phi))
    
    y_comp = (result[0] * np.sin(theta) * np.sin(phi) + 
                result[1] * np.cos(theta) * np.sin(phi) + 
                result[2] * np.cos(phi))
    
    z_comp = (result[0] * np.cos(theta) - 
            result[1] * np.sin(theta))
    
    mlab.clf() # clears the mayavi screen if one is already open
    mlab.view(azimuth=view[0], elevation=view[1])
    if points:
        scatter_plot = mlab.points3d(x, y, z, scale_factor=0.01) # creates a set of discrete points
    if vectors:
        vector_plot = mlab.quiver3d(x, y, z, x_comp, y_comp, z_comp, scale_factor=1, mode='arrow') #
    if mesh:
        mesh_plot = mlab.mesh(x, y, z, color=(0.0, 0.0, 0.2))
    for i in range(len(time)):
        x_comp_new = x_comp * periodic_f[i]
        
        y_comp_new = y_comp * periodic_f[i]

        z_comp_new = z_comp * periodic_f[i]
        if vectors:
            vector_plot.mlab_source.set(u = x_comp_new, 
                                        v = y_comp_new, 
                                        w = z_comp_new)
        if mesh:
            mesh_plot.mlab_source.set(x = x + x_comp_new, 
                                y = y + y_comp_new,
                                z = z + z_comp_new)
        if points:
            scatter_plot.mlab_source.set(x = x + x_comp_new, 
                                y = y + y_comp_new,
                                z = z + z_comp_new)
        if savefig==False:
                mlab.process_ui_events()
        if savefig:
            mlab.savefig(f'ani_{i}.png')
            images.append(imageio.imread(f'ani_{i}.png'))
    
    if savefig:
        imageio.mimsave(f'{save_path}/S^{m}_{l}_surf.gif', images, loop=0)
        [os.remove(f) for f in os.listdir(save_path) if f.endswith('.png') and 'ani_' in f] # removes png files after creating gif

In [13]:
# an example showing the usage of the above function

surf_anim_S(0, 2, phi_no=20, theta_no=30, time_max=4, 
            time_interval=0.01, vectors=True, points=False)

In [14]:
# an example showing how to use the above function to save the output as a gif

# NOTE: details of the animation such as the background color, the size of the figure, etc. 
#       can be changed in the mayavi window
#       to do this, first create an animation with savefig = False, adjust the appearance, and then run the code again with savefig = True
surf_anim_S(0, 2, phi_no=20, theta_no=30, time_max=4, 
            time_interval=0.05, save_path='./', savefig=True, view=(0, 90))

### Visualising the Toroidal modes (surface only)

In [15]:
def surf_anim_T(m, l, phi_no, theta_no, time_max, time_interval, omega=1, shape='sphere', savefig=False, points=True, mesh=True, vectors=False, save_path='./', view=(0, 90)):
    # phi_no and theta_no is the desired resoluion of the animated mesh in the azimuthal and polar directions respectively
    # time_max is the total time of the animation
    # time_interval is the resolution of the time axis
    # omega (default = 1) is the frequency of the animation (the eigenfrequency of the mode can be used but it is usually not suited for visualisation)
    # shape: sphere, hemisphere, 3/4th of a sphere, 1/4th of a sphere
    # savefig: whether or not the animation is to be saved
    # save_path: the path at which to save the .gif for the animation as well as the images for all the frames (temporarily)
    # view = (azimuthal angle, elevation angle): allows one to set the viewing angle for the animation, which can also be adjusted manually in the mayavi window
    # NOTE: if savefig is set to true, the function will create each frame as an image and stitch them all together as a gif, deleting the images, this may take up a lot of space and time for longer or smoother animations

    if savefig:
        images=[]
    
    # creating the mesh on which to calculate the surface vectors        
    if shape=='sphere':
        phi, theta = np.mgrid[0.0001:2.0*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='hemisphere':
        phi, theta = np.mgrid[0.0001:1.0*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    elif shape=='3_quarter':
        phi, theta = np.mgrid[0.0001:1.5*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    elif shape=='1_quarter':
        phi, theta = np.mgrid[0.0001:0.5*np.pi-0.0001:phi_no*1j, 0.0001:np.pi-0.0001:theta_no*1j] 
    # creating the time axis for the animation
    time = np.arange(0, time_max, time_interval)
    
    # calculating the x, y, z coordinates of each point 
    x = np.sin(theta) * np.cos(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(theta)

    # calculating the output of the surface vectors
    T = T_(m, l, phi, theta)
    result = T
    
    # the periodic function which introduces the time dependence
    # NOTE: we are only taking the real part of this, so it can be replaced with a cosine
    periodic_f = np.real(np.e**(1j * time) * omega)

    # calculating the x, y, z components as the points are plotted in cartesian coordinates
    x_comp = (result[0] * np.sin(theta) * np.cos(phi) + 
                result[1] * np.cos(theta) * np.cos(phi) - 
                result[2] * np.sin(phi))
    
    y_comp = (result[0] * np.sin(theta) * np.sin(phi) + 
                result[1] * np.cos(theta) * np.sin(phi) + 
                result[2] * np.cos(phi))
    
    z_comp = (result[0] * np.cos(theta) - 
            result[1] * np.sin(theta))
    
    mlab.clf() # clears the mayavi screen if one is already open
    mlab.view(azimuth=view[0], elevation=view[1])
    if points:
        scatter_plot = mlab.points3d(x, y, z, scale_factor=0.05) # creates a set of discrete points
    if vectors:
        vector_plot = mlab.quiver3d(x, y, z, x_comp, y_comp, z_comp, scale_factor=1, mode='arrow') 
    if mesh:
        mesh_plot = mlab.mesh(x, y, z, color=(0.0, 0.0, 0.2))

    for i in range(len(time)): # animation function.  This is called sequentially
        x = np.sin(theta + (T[1] * periodic_f[i])) * np.cos(phi + (T[2] * periodic_f[i]))
        y = np.sin(theta + (T[1] * periodic_f[i])) * np.sin(phi + (T[2] * periodic_f[i]))
        z = np.cos(theta + (T[1] * periodic_f[i]))
        
        if points:
            scatter_plot.mlab_source.set(x = x, y= y, z = z)
        if vectors:
            x_comp_new = x_comp * periodic_f[i]
            y_comp_new = y_comp * periodic_f[i]
            z_comp_new = z_comp * periodic_f[i]
            vector_plot.mlab_source.set(u = x_comp_new,
                                        v = y_comp_new,
                                        w = z_comp_new)
        if mesh:
            mesh_plot.mlab_source.set(x = x, y= y, z = z)
        if savefig==False:
                mlab.process_ui_events()
        if savefig:
            mlab.savefig(f'ani_{i}.png')
            images.append(imageio.imread(f'ani_{i}.png'))
    
    if savefig:
        imageio.mimsave(f'{save_path}/T^{m}_{l}_surf.gif', images, loop=0)
        [os.remove(f) for f in os.listdir(save_path) if f.endswith('.png') and 'ani_' in f] # removes png files after creating gif

In [16]:
# an example showing the usage of the above function

surf_anim_T(0, 2, phi_no=20, theta_no=75, time_max=4, time_interval=0.01, points=True, mesh=True)

In [17]:
# an example showing the usage of the above function to save an animation

surf_anim_T(0, 2, phi_no=20, theta_no=30, time_max=4, time_interval=0.05, points=True, mesh=True, savefig=True)

### Reading the Mineos output

The following function can be used to read the plain output text file generated by Mineos which contains the properties of all calculated normal modes. It returns a dictionary containing the eigenfrequencies for all modes present in the output file.  

In [18]:
# this function read the
def read_plain_for_eigfreq(path_to_out_file):
    # each Mineos output plain file contains properties of the normal modes starting from the 197th line
    output = np.genfromtxt(path_to_out_file, skip_header=196, dtype=str) 
    dictionary = dict()
    for i in range(len(output)):
        dictionary[f'{output[i, 1].upper() + '^' + output[i, 0] + '_' + output[i, 2]}'] = np.float64(output[i, 4]) / 1000
    return dictionary

In [19]:
test_S_plain = read_plain_for_eigfreq('./test_S_out_plain')
test_T_plain = read_plain_for_eigfreq('./test_T_out_plain')

The following function can be used to read all of the output files created by the eigen2asc utility in Mineos that are present in the destination folder. It creates a dictionary containing the eigenfunctions for all modes for which output files are present. 

In [20]:
def read_eig(path_to_directory):
    dictionary = dict()
    for filename in os.listdir(path_to_directory):
        stripped = filename.rstrip('.ASC')
        split = filename.split(sep='.')
        mode = split[0]
        norder = int(split[1])
        lorder = int(split[2])
        dictionary['mode'] = mode
        dictionary[f'{norder}_{mode}_{lorder}'] = np.loadtxt(os.path.join(path_to_directory, filename), skiprows=1)
    return dictionary

In [21]:
# an example showing how the read_eig function can be used with the test directory containing Toroidal eigenfunctions
test_T_dict = read_eig('./test_T_asc')
test_S_dict = read_eig('./test_S_asc')

In [ ]:
# an example showing how an individual eigenfunction can be obtained from the dictionary
# format: 'N_MODE_L' (where MODE = S/T)
print(test_T_dict['0_T_2'])
T_0_2_eig = test_T_dict['0_T_2']

In [23]:
# this function plots the eigenfunctions for each mode from a dictionary
# eig_dict: the dictionary made using the read_eig() function
# key: if NONE, plots all of the eigenfunctions. 
#      if given, makes the plot for that particular mode.
#      format = 'N_MODE_L' where MODE = S/T

def plot_eig(eig_dict, key=None):
    # plots all eigenfunctions
    if key == None:
        keys = list(eig_dict.keys()) # stores names of all modes for easy reference
        plt.close() # closes any previous figures to prevent lag
        
        if eig_dict['mode'] == 'T':
            fig = plt.figure(constrained_layout=True, figsize=(3, 3*len(keys)))
            subplots = fig.subplots(nrows=len(eig_dict.keys()) - 1, ncols=1, sharex='col') # creating subplots for different modes
            for row, ax in enumerate(subplots):
                ax.set_title(f'{keys[row + 1]}') # each mode gets appropriate title
                ax.xaxis.set_tick_params(which='both', labelbottom=True) # for making the tick labels appear on all subplots
                ax.axvline(0, ls='--', alpha=0.5, color='k', label='0')
                ax.axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer core')
                ax.plot((eig_dict[keys[row + 1]])[:, 1], (eig_dict[keys[row + 1]])[:, 0]) # makes the plot
                ax.legend()

        if eig_dict['mode'] == 'S':
            fig = plt.figure(constrained_layout=True, figsize=(7, 3*len(keys)))
            subplots = fig.subplots(nrows=len(eig_dict.keys()) - 1, ncols=2, sharex='col') # creating subplots for different modes
            for row, ax in enumerate(subplots):
                ax[0].set_title(f'{keys[row + 1]} - U') # each mode gets appropriate title
                ax[0].xaxis.set_tick_params(which='both', labelbottom=True) # for making the tick labels appear on all subplots
                ax[0].axvline(0, ls='--', alpha=0.5, color='k')
                ax[0].axhline(5961*1e3, ls='--', alpha=0.5, color='darkgoldenrod', label='Mantle')
                ax[0].axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer Core')
                ax[0].axhline(1221*1e3, ls='--', alpha=0.5, color='navy', label='Inner Core')
                ax[0].plot((eig_dict[keys[row + 1]])[:, 1], (eig_dict[keys[row + 1]])[:, 0]) # makes the plot
                ax[0].legend()

                ax[1].set_title(f'{keys[row + 1]} - V') # each mode gets appropriate title
                ax[1].xaxis.set_tick_params(which='both', labelbottom=True) # for making the tick labels appear on all subplots
                ax[1].axvline(0, ls='--', alpha=0.5, color='k')
                ax[1].axhline(5961*1e3, ls='--', alpha=0.5, color='darkgoldenrod', label='Mantle')
                ax[1].axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer Core')
                ax[1].axhline(1221*1e3, ls='--', alpha=0.5, color='navy', label='Inner Core')
                ax[1].plot((eig_dict[keys[row + 1]])[:, 3], (eig_dict[keys[row + 1]])[:, 0]) # makes the plot
                ax[1].legend()

# executed if a mode is provided as the key, plots a specific mode
    else:
        if eig_dict['mode'] == 'T':
            fig = plt.figure(figsize=(3, 3))
            plt.title(f'{key}') # each mode gets appropriate title
            plt.axvline(0, ls='--', alpha=0.5, color='k', label='0')
            plt.axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer-Core')
            plt.plot((eig_dict[key])[:, 1], (eig_dict[key])[:, 0]) # makes the plot
            plt.legend()

        if eig_dict['mode'] == 'S':
            fig = plt.figure(constrained_layout=True, figsize=(6, 3))
            subplots = fig.subplots(nrows=1, ncols=2, sharex='row') # creating subplots for different modes
            for row, ax in enumerate(subplots):
                if row == 0:
                    ax.set_title(f'{key} - U') # each mode gets appropriate title
                    ax.xaxis.set_tick_params(which='both', labelbottom=True) # for making the tick labels appear on all subplots
                    ax.axvline(0, ls='--', alpha=0.5, color='k')
                    ax.axhline(5961*1e3, ls='--', alpha=0.5, color='darkgoldenrod', label='Mantle')
                    ax.axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer Core')
                    ax.axhline(1221*1e3, ls='--', alpha=0.5, color='navy', label='Inner Core')
                    ax.plot((eig_dict[key])[:, 1], (eig_dict[key])[:, 0]) # makes the plot
                    ax.legend()
                if row == 1:
                    ax.set_title(f'{key} - V') # each mode gets appropriate title
                    ax.xaxis.set_tick_params(which='both', labelbottom=True) # for making the tick labels appear on all subplots
                    ax.axvline(0, ls='--', alpha=0.5, color='k')
                    ax.axhline(5961*1e3, ls='--', alpha=0.5, color='darkgoldenrod', label='Mantle')
                    ax.axhline(3482*1e3, ls='--', alpha=0.5, color='g', label='Outer Core')
                    ax.axhline(1221*1e3, ls='--', alpha=0.5, color='navy', label='Inner Core')
                    ax.plot((eig_dict[key])[:, 3], (eig_dict[key])[:, 0]) # makes the plot
                    ax.legend()
    plt.show()

In [ ]:
# an example showing the plots for Toroidal eigenfunctions
plot_eig(test_T_dict)

In [ ]:
# an example showing the usage of the key parameter
plot_eig(test_T_dict, key='0_T_2')

In [ ]:
# an example showing the plots for Spheroidal eigenfunctions
plot_eig(test_S_dict)

### Visualising Spheroidal Modes for a complete Earth

In [27]:
def anim_S(m, l, n, phi_no, theta_no, time_max, time_interval, eigen_dict, out_plain, shape='sphere', r_points='layers', amplify_factor=1000, speedup_factor = 1e4, mesh=True, points=True, vectors=False, savefig=False, savepath='./', view=(0, 90)):
    # phi_no & theta_no: resolution of the mesh in the phi and theta directions
    # time_max: duration of the animation
    # eigen_dict: the dictionary containing eigenfunctions obtained using the read_eig() function
    # out_plain: the dictionary containing the eigenfrequencies obtained using the read_plain_for_eigfreq() function
    # amplify_factor: the factor by which the animation is amplified to make the displacements noticeable
    # speedup_factor: the factor by which the animation is sped up
    # r_points (can be an int or 'layers'): the number of layers to be displayed. 
    #           if an int, this many equally spaced values are chosen for the radius
    #           if 'layers', the different layers of the Earth are shown as follows:
    #             Crust = 6371 km 
    #             Mantle = 5961 km
    #             Outer core = 3,482 km
    #             Inner core = 1221 km
    
    eigen_key = f'{n}_{eigen_dict['mode']}_{l}'
    out_key = f'{eigen_dict['mode']}^{m}_{l}'
    
    eigen_db = eigen_dict[eigen_key]
    omega = out_plain[out_key]
    time = np.arange(0, time_max, time_interval)

    # for saving the animation
    if savefig:
        images=[]
        
    if shape=='sphere':
        phi, theta = np.mgrid[0.001:2.0*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='hemisphere':
        phi, theta = np.mgrid[0.001:1.0*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='3_quarter':
        phi, theta = np.mgrid[0.001:1.5*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='1_quarter':
        phi, theta = np.mgrid[0.001:0.5*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    
    if type(r_points) == int:
        indices = np.zeros(r_points)
        for i in range(r_points):
            indices[i] = i * int(len(eigen_db[:, 0])/r_points)
        radius = np.zeros_like(indices)
        for i in range(len(radius)):
            radius[i] = eigen_db[int(indices[i]), 0]
    
    elif r_points == 'layers':
        indices = np.array([0, 45, 118, 151])
        radius = np.zeros_like(indices)
        for i in range(len(radius)):
            radius[i] = eigen_db[int(indices[i]), 0]

    U = np.zeros_like(radius, dtype=np.float64)
    V = np.zeros_like(radius, dtype=np.float64)

    for i in range(len(U)):
        U[i] = eigen_db[int(indices[i]), 1]
        V[i] = eigen_db[int(indices[i]), 3]

    x = np.zeros((len(radius), ) + np.shape(phi))
    y = np.zeros_like(x)
    z = np.zeros_like(x)

    for i in range(len(radius)):
        x[i] = np.sin(theta) * np.cos(phi) * radius[i]
        y[i] = np.sin(theta) * np.sin(phi) * radius[i]
        z[i] = np.cos(theta) * radius[i]

    S = S_(m, l, phi, theta)
    R = R_(m, l, phi, theta)

    periodic_f = np.real(np.e**(1j * time) * omega)

    x_comp = np.zeros_like(x)
    y_comp = np.zeros_like(x)
    z_comp = np.zeros_like(x)
    
    for i in range(len(radius)):
        x_comp[i] =( (S[0] * np.sin(theta) * np.cos(phi)  + 
                    S[1] * np.cos(theta) * np.cos(phi) - 
                    S[2] * np.sin(phi)) * V[i]
                    + 
                (R[0] * np.sin(theta) * np.cos(phi)  + 
                    R[1] * np.cos(theta) * np.cos(phi) - 
                    R[2] * np.sin(phi)) * U[i] )
        
        y_comp[i] = ( (S[0] * np.sin(theta) * np.sin(phi)  + 
                    S[1] * np.cos(theta) * np.sin(phi) + 
                    S[2] * np.cos(phi)) * V[i]
                    + 
                    (R[0] * np.sin(theta) * np.sin(phi)  + 
                    R[1] * np.cos(theta) * np.sin(phi) + 
                    R[2] * np.cos(phi)) * U[i])
        
        z_comp[i] = ((S[0] * np.cos(theta)  - 
            S[1] * np.sin(theta)) * V[i]
            +
            (R[0] * np.cos(theta)  - 
            R[1] * np.sin(theta)) * U[i]
            )
    mlab.clf()
    mlab.view(azimuth=view[0], elevation=view[1])
    
    if points:
        spheres = [mlab.points3d(x[0], y[0], z[0], scale_factor = 80000)]
    if mesh:
        meshes = [mlab.mesh(x[0], y[0], z[0], color=(0.2, 0.2, 0.4))]
    if vectors:
        vector_plots = [mlab.quiver3d(x[0], y[0], z[0], x_comp[0], y_comp[0], z_comp[0], scale_factor = 1)]
    for i in range(1, len(radius)):
        if points:
            spheres.append(mlab.points3d(x[i], y[i], z[i], scale_factor= 80000))
        if mesh:
            meshes.append(mlab.mesh(x[i], y[i], z[i], color=(0.2, 0.2, 0.4)))
        if vectors:
            vector_plots.append(mlab.quiver3d(x[i], y[i], z[i], x_comp[i], y_comp[i], z_comp[i], scale_factor = 1))
    
    for i in range(len(time)):
        # update the x, y and z components for all points
        x_comp_new = amplify_factor * x_comp * periodic_f[i]
        y_comp_new = amplify_factor * y_comp * periodic_f[i]
        z_comp_new = amplify_factor * z_comp * periodic_f[i]
        
        # updating each mesh on the plot
        for j in range(len(radius)):
            if points:
                spheres[j].mlab_source.set(x = x[j] + x_comp_new[j], 
                                          y = y[j] + y_comp_new[j], 
                                          z = z[j] + z_comp_new[j])
            if mesh:
                meshes[j].mlab_source.set(x = x[j] + x_comp_new[j], 
                                          y = y[j] + y_comp_new[j], 
                                          z = z[j] + z_comp_new[j])
            if vectors:
                vector_plots[j].mlab_source.set(u = x_comp_new[j], 
                                                v = y_comp_new[j], 
                                                w = z_comp_new[j])

            if not savefig:
                mlab.process_ui_events()
        if savefig:
            mlab.savefig(f'ani_{i}.png')
            images.append(imageio.imread(f'ani_{i}.png'))
    if savefig:
        imageio.mimsave(f'{savepath}/{n}_S^{m}_{l}.gif', images, loop=0)
        [os.remove(f) for f in os.listdir(f'{savepath}') if f.endswith('.png') and 'ani_' in f] # removes png files after creating gif


In [28]:
# an example showing the usage of the anim_S function
# NOTE: savefig can be set to True to save the figure

anim_S(0, 2, 0, 20, 20, time_max=2, time_interval=0.1, 
       eigen_dict=test_S_dict, out_plain=test_S_plain, 
       amplify_factor=2*1e9, speedup_factor=1e6, 
       mesh=True, vectors=False, shape='hemisphere', 
       r_points='layers', view=(90, 90), savefig=False)

### Visualising Toroidal Modes for a complete Earth

In [29]:
def anim_T(m, l, n, phi_no, theta_no, time_max, time_interval, eigen_dict, out_plain, shape='sphere', r_points='layers', amplify_factor=1000, speedup_factor = 1e4, mesh=True, points=True, vectors=False, savefig=False, savepath='./', view=(0, 90)):
    # phi_no & theta_no: resolution of the mesh in the phi and theta directions
    # time_max: duration of the animation
    # eigen_dict: the dictionary containing eigenfunctions obtained using the read_eig() function
    # out_plain: the dictionary containing the eigenfrequencies obtained using the read_plain_for_eigfreq() function
    # amplify_factor: the factor by which the animation is amplified to make the displacements noticeable
    # speedup_factor: the factor by which the animation is sped up
    # r_points (can be an int or 'layers'): the number of layers to be displayed. 
    #           if an int, this many equally spaced values are chosen for the radius
    #           if 'layers', the different layers of the Earth are shown as follows:
    #             Crust = 6371 km 
    #             Mantle = 5961 km
    #             Outer core = 3,482 km
    #             Inner core = 1221 km
    
    eigen_key = f'{n}_{eigen_dict['mode']}_{l}'
    out_key = f'{eigen_dict['mode']}^{m}_{l}'
    
    eigen_db = eigen_dict[eigen_key]
    omega = out_plain[out_key]
    time = np.arange(0, time_max, time_interval)

    # for saving the animation
    if savefig:
        images=[]
        
    if shape=='sphere':
        phi, theta = np.mgrid[0.001:2.0*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='hemisphere':
        phi, theta = np.mgrid[0.001:1.0*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='3_quarter':
        phi, theta = np.mgrid[0.001:1.5*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    elif shape=='1_quarter':
        phi, theta = np.mgrid[0.001:0.5*np.pi-0.001:phi_no*1j, 0.001:np.pi-0.001:theta_no*1j] # phi = azimuthal, theta = polar 
    
    if type(r_points) == int:
        indices = np.zeros(r_points)
        for i in range(r_points):
            indices[i] = i * int(len(eigen_db[:, 0])/r_points)
        radius = np.zeros_like(indices)
        for i in range(len(radius)):
            radius[i] = eigen_db[int(indices[i]), 0]
    
    elif r_points == 'layers':
        indices = np.array([0, 45, 118, 151])
        radius = np.zeros_like(indices)
        for i in range(len(radius)):
            radius[i] = eigen_db[int(indices[i]), 0]

    W = np.zeros_like(radius, dtype=np.float64)

    for i in range(len(W)):
        W[i] = eigen_db[int(indices[i]), 1]

    x = np.zeros((len(radius), ) + np.shape(phi))
    y = np.zeros_like(x)
    z = np.zeros_like(x)

    for i in range(len(radius)):
        x[i] = np.sin(theta) * np.cos(phi) * radius[i]
        y[i] = np.sin(theta) * np.sin(phi) * radius[i]
        z[i] = np.cos(theta) * radius[i]

    T = T_(m, l, phi, theta)
    periodic_f = np.real(np.e**(1j * time) * omega)

    x_comp = np.zeros_like(x)
    y_comp = np.zeros_like(x)
    z_comp = np.zeros_like(x)
    
    for i in range(len(radius)):
        x_comp[i] =(T[0] * np.sin(theta) * np.cos(phi) + 
                    T[1] * np.cos(theta) * np.cos(phi) - 
                    T[2] * np.sin(phi)) * W[i] 
        
        y_comp[i] = (T[0] * np.sin(theta) * np.sin(phi) + 
                    T[1] * np.cos(theta) * np.sin(phi) + 
                    T[2] * np.cos(phi)) * W[i] 
        
        z_comp[i] = (T[0] * np.cos(theta) - 
            T[1] * np.sin(theta)) * W[i] 
        
    mlab.clf()
    mlab.view(azimuth=view[0], elevation=view[1])
    
    if points:
        spheres = [mlab.points3d(x[0], y[0], z[0], scale_factor = 80000)]
    if mesh:
        meshes = [mlab.mesh(x[0], y[0], z[0], color=(0.2, 0.2, 0.4))]
    if vectors:
        vector_plots = [mlab.quiver3d(x[0], y[0], z[0], x_comp[0], y_comp[0], z_comp[0], scale_factor = 1)]
    for i in range(1, len(radius)):
        if points:
            spheres.append(mlab.points3d(x[i], y[i], z[i], scale_factor= 80000))
        if mesh:
            meshes.append(mlab.mesh(x[i], y[i], z[i], color=(0.2, 0.2, 0.4)))
        if vectors:
            vector_plots.append(mlab.quiver3d(x[i], y[i], z[i], x_comp[i], y_comp[i], z_comp[i], scale_factor = 1))
    
    for i in range(len(time)):
        # update the x, y and z components for all points
        x_comp_new = amplify_factor * x_comp * periodic_f[i]
        y_comp_new = amplify_factor * y_comp * periodic_f[i]
        z_comp_new = amplify_factor * z_comp * periodic_f[i]
        
        # updating each mesh on the plot
        for j in range(len(radius)):
            if points:
                spheres[j].mlab_source.set(x = x[j] + x_comp_new[j], 
                                          y = y[j] + y_comp_new[j], 
                                          z = z[j] + z_comp_new[j])
            if mesh:
                meshes[j].mlab_source.set(x = x[j] + x_comp_new[j], 
                                          y = y[j] + y_comp_new[j], 
                                          z = z[j] + z_comp_new[j])
            if vectors:
                vector_plots[j].mlab_source.set(u = x_comp_new[j], 
                                                v = y_comp_new[j], 
                                                w = z_comp_new[j])

            if not savefig:
                mlab.process_ui_events()
        if savefig:
            mlab.savefig(f'ani_{i}.png')
            images.append(imageio.imread(f'ani_{i}.png'))
    if savefig:
        imageio.mimsave(f'{savepath}/{n}_T^{m}_{l}.gif', images, loop=0)
        [os.remove(f) for f in os.listdir(f'{savepath}') if f.endswith('.png') and 'ani_' in f] # removes png files after creating gif


In [31]:
# an example showing the usage of the anim_T() function
# NOTE: savefig can be set to True to save the figure

anim_T(0, 2, 0, 20, 30, time_max=8, time_interval=0.1, 
       eigen_dict=test_T_dict, out_plain=test_T_plain,
       amplify_factor=1e10,
       speedup_factor=1e6, mesh=True, vectors=False, 
       shape='sphere', r_points='layers', view=(90, 90), 
       savefig=False)